In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

from tqdm import tqdm

/home/msst/Utils/miniconda3/envs/qenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BITS = 2
GROUP_SIZE = 64

In [3]:
# FP
path_to_model = "/media/msst/ssd_storage1/ml/llm/pretrained_models/Llama2-7B"
tokenizer = AutoTokenizer.from_pretrained(path_to_model)
# model = AutoModelForCausalLM.from_pretrained(
model = qlib._modeling.modeling_llama.LlamaForCausalLM.from_pretrained(
    path_to_model,
    device_map="cpu",
    dtype="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 85.13it/s]


In [4]:
class QuantLinear(nn.Module):
    def __init__(self, weight_shape, group_size):
        super().__init__()
        self.weight_shape = weight_shape
        self.group_size = group_size
        self.scale_size = [weight_shape[0], weight_shape[1] // group_size]
        self.scale = nn.Parameter(torch.empty(self.scale_size))
        self.offset = nn.Parameter(torch.empty(self.scale_size))
        self.register_buffer(
            "compressed_weight",
            torch.empty(
                weight_shape,
                dtype=torch.uint8,
                requires_grad=False
            )
        )

    def reshape_weight_for_scaling(self, w):
        return w.reshape(
            self.weight_shape[0], self.weight_shape[1] // self.group_size, -1
        )


    @torch.compile()
    def reconstruct_weight(self):
        w = self.reshape_weight_for_scaling(self.compressed_weight)
        w = w * self.scale[..., None] - self.offset[..., None]
        w = w.reshape(self.weight_shape)
        return w


    def forward(self, x):
        w = self.reconstruct_weight()
        return torch.nn.functional.linear(x, w.to(x.dtype))

In [5]:
# model = AutoModelForCausalLM.from_pretrained(
qmodel = qlib._modeling.modeling_llama.LlamaForCausalLM.from_pretrained(
    path_to_model,
    device_map="cpu",
    dtype="auto"
)

def wrap_model(current_module, prefix=''):
    for module_name, module in current_module.named_children():
        full_name = f"{prefix}.{module_name}" if prefix else module_name
        
        if "proj" in module_name:
            weight_shape = module.weight.data.shape
            setattr(current_module, module_name, QuantLinear(weight_shape, GROUP_SIZE))
        else:
            wrap_model(module, full_name)

wrap_model(qmodel)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 109.44it/s]


In [6]:
class MinMaxInitializer:
    def __init__(self):
        pass

    @torch.no_grad()
    def __call__(self, x_grouped, negative_clip, positive_clip):
        x_min = x_grouped.min(axis=-1)[0].unsqueeze(-1).float()
        x_max = x_grouped.max(axis=-1)[0].unsqueeze(-1).float()

        offset = (x_max * negative_clip - x_min * positive_clip) / (positive_clip - negative_clip)
        scale = (x_max + offset) / positive_clip
        scale = torch.abs(scale)

        scale = scale.reshape(x_grouped.shape[0], x_grouped.shape[1])
        offset = offset.reshape(x_grouped.shape[0], x_grouped.shape[1])

        return scale.contiguous(), offset.contiguous()

initializer = MinMaxInitializer()

@torch.no_grad()
def configure_single_layer(qlayer, layer, bits, C=None):
    max_int_val = 2**bits - 1

    orig_weight = layer.weight.data
    if C is not None:
        orig_weight = C.float() @ orig_weight.float()

    orig_weight_reshaped = qlayer.reshape_weight_for_scaling(orig_weight)
    scale, offset = initializer(orig_weight_reshaped, negative_clip=0, positive_clip=max_int_val)
    
    quant_weight = (orig_weight_reshaped + offset[..., None]) / scale[..., None]
    quant_weight = quant_weight.reshape_as(orig_weight)

    quant_weight = torch.clamp(torch.round(quant_weight), 0, max_int_val).to(torch.uint8)

    qlayer.compressed_weight.copy_(quant_weight)
    qlayer.scale.copy_(scale)
    qlayer.offset.copy_(offset)


@torch.no_grad()
def init_quant_model(qmodel, model, bits):
    for qmodule_name, qmodule in qmodel.named_modules():
        if isinstance(qmodule, QuantLinear):
            orig_module = model.get_submodule(qmodule_name)
            configure_single_layer(qmodule, orig_module, bits)

            err = torch.mean(((orig_module.weight.data.cpu() - qmodule.reconstruct_weight().cpu()) / (orig_module.weight.data.cpu().std() + 1e-8))**2)
            print(err, qmodule_name)


# init_quant_model(qmodel, model, bits=BITS)

In [7]:
path = f"/home/msst/repo/Quantization/nb/adaptive_rounding/init_w{BITS}gs{GROUP_SIZE}.pth"
# torch.save(qmodel.state_dict(), path)
qmodel.load_state_dict(torch.load(path))

<All keys matched successfully>

In [8]:
from transformers.masking_utils import create_causal_mask


@torch.no_grad()
def prepare_hessian(activations):
    hidden_size = activations[0].shape[-1]
    H = torch.zeros(hidden_size, hidden_size).cuda()
    for act in activations:
        act = act.cuda().view(-1, act.shape[-1]).float() / hidden_size ** 0.5
        H += act.T @ act
    return H


@torch.no_grad()
def prepare_hessian_q(activations, activations_q):
    hidden_size = activations[0].shape[-1]
    H_q = torch.zeros(hidden_size, hidden_size).cuda()
    for act_id in range(len(activations)):
        act = activations[act_id].cuda().view(-1, hidden_size).float() / hidden_size ** 0.5
        act_q = activations_q[act_id].cuda().view(-1, hidden_size).float() / hidden_size ** 0.5
        H_q += act.T @ act_q
    return H_q


def prepare_C(H, Hq):
    I = torch.eye(H.shape[0]).cuda() * 0 #* H.std()
    C = torch.linalg.inv(H + I) @ Hq
    return C


def hessian_loss(layer_q, layer_fp, H, C):
    w_q = layer_q.reconstruct_weight()
    if C is not None:
        w_q = w_q @ C.T
    w = layer_fp.weight
    delta_w = layer_q.reconstruct_weight() - layer_fp.weight
    return torch.trace(delta_w @ H @ delta_w.T)


def optimize_quant_params(
        layer_q,
        layer_fp,
        bits,
        H,
        C=None,
    ):
    trainable_params = [layer_q.scale, layer_q.offset]    
    optim = torch.optim.Adam(trainable_params, lr=1e-3)
    n_steps = 100
    
    with torch.amp.autocast('cuda', dtype=torch.bfloat16):
        for i in range(n_steps):
            optim.zero_grad()
            loss = hessian_loss(layer_q, layer_fp, H, C)
            if i == 0:
                init_loss =  loss.item()
            loss.backward()
            optim.step()

        print(f"{init_loss} -> {loss}")


def hessian_loss_ste(layer_q, layer_fp, H, bits):
    max_int_val = 2**bits - 1

    latent_weight_reshaped = layer_q.reshape_weight_for_scaling(layer_fp.weight + layer_q.weight_addition)
    latent_weight_scaled = (latent_weight_reshaped + layer_q.offset[..., None]) / layer_q.scale[..., None]

    quant_weight = torch.clamp(torch.round(latent_weight_scaled), 0, max_int_val).to(torch.uint8)
    quant_weight_ste = quant_weight + latent_weight_scaled

    layer_q.compressed_weight.copy_(quant_weight.reshape_as(layer_fp.weight))

    weight_reco = quant_weight_ste * layer_q.scale[..., None] - layer_q.offset[..., None]
    weight_reco = weight_reco.reshape_as(layer_fp.weight)

    delta_w = weight_reco - layer_fp.weight

    C = 1e-6
    return torch.trace(delta_w @ H @ delta_w.T) + C * torch.sum(layer_q.weight_addition ** 2)


def optimize_quant_params_ste(
        layer_q,
        layer_fp,
        bits,
        H
    ):
    layer_q.weight_addition = nn.Parameter(torch.zeros_like(layer_fp.weight.data).float())

    # trainable_params = [layer_q.scale, layer_q.offset, layer_q.latent_weight]        
    # trainable_params = [layer_q.scale, layer_q.offset]
    trainable_params = [layer_q.weight_addition]        
    optim = torch.optim.Adam(trainable_params, lr=1e-4)
    n_steps = 100

    with torch.amp.autocast('cuda', dtype=torch.bfloat16):
        for i in range(n_steps):
            optim.zero_grad()

            loss = hessian_loss_ste(layer_q, layer_fp, H, bits)
            if i == 0:
                init_loss =  loss.item()
            loss.backward()
            optim.step()

        print(f"{init_loss} -> {loss}")

    del layer_q.weight_addition


def init_quant_block_hessian(
        block_q,
        block_fp,
        bits,
        activations,
        causal_mask,
        position_embeddings,
        with_opt=True,
        ):

    ##### Attention #####

    # Copy activations for the residual stream
    residual_activations = [x.clone() for x in activations]

    # Collect activations after input_layernorm
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            act = block_fp.input_layernorm(act)
            activations[act_id] = act.cpu()

    # Initialize q,k,v-projs
    H = prepare_hessian(activations)
    block_q_attn = block_q.self_attn
    block_fp_attn = block_fp.self_attn
    for layer_name in ["q_proj", "k_proj", "v_proj"]:
        layer_q = getattr(block_q_attn, layer_name)
        layer_fp = getattr(block_fp_attn, layer_name)
        configure_single_layer(layer_q, layer_fp, bits)
        if with_opt:
            optimize_quant_params(layer_q, layer_fp, bits, H)

    # Collect attention-out activations
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            act = block_fp_attn.compute_attention(
                hidden_states=act, 
                position_embeddings=position_embeddings,
                attention_mask=causal_mask
            )[0]
            activations[act_id] = act.cpu()

    # Initialize o_proj
    layer_q = block_q.self_attn.o_proj
    layer_fp = block_fp.self_attn.o_proj
    H = prepare_hessian(activations)
    configure_single_layer(layer_q, layer_fp, bits)
    if with_opt:
        optimize_quant_params(layer_q, layer_fp, bits, H)

    # Collect self_attn outs
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            res_act = residual_activations[act_id].cuda()
            act = block_fp.self_attn.o_proj(act)
            activations[act_id] = (act + res_act).cpu()

    ##### MLP #####

    # Copy activations for the residual stream
    residual_activations = [x.clone() for x in activations]

    # Collect activations after post_attention_layernorm
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            act = block_fp.post_attention_layernorm(act)
            activations[act_id] = act.cpu()

    # Initialize gate_proj and up_proj
    H = prepare_hessian(activations)
    block_q_mlp = block_q.mlp
    block_fp_mlp = block_fp.mlp
    for layer_name in ["gate_proj", "up_proj"]:
        layer_q = getattr(block_q_mlp, layer_name)
        layer_fp = getattr(block_fp_mlp, layer_name)
        configure_single_layer(layer_q, layer_fp, bits)
        if with_opt:
            optimize_quant_params(layer_q, layer_fp, bits, H)

    # Collect internal mlp activations
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            act = block_fp_mlp.act_fn(block_fp.mlp.gate_proj(act)) * block_fp.mlp.up_proj(act)
            activations[act_id] = act.cpu()

    # Initialize down_proj
    layer_q = block_q.mlp.down_proj
    layer_fp = block_fp.mlp.down_proj
    H = prepare_hessian(activations)
    configure_single_layer(layer_q, layer_fp, bits)
    if with_opt:
        optimize_quant_params(layer_q, layer_fp, bits, H)

    # Collect mlp outs
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            res_act = residual_activations[act_id].cuda()
            act = block_fp.mlp.down_proj(act)
            activations[act_id] = (act + res_act).cpu()


def init_quant_block_hessian_2(
        block_q,
        block_fp,
        bits,
        activations,
        activations_q,
        causal_mask,
        position_embeddings,
        with_opt=True,
        ):

    ##### Attention #####

    # Copy activations for the residual stream
    residual_activations = [x.clone() for x in activations]
    residual_activations_q = [x.clone() for x in activations_q]

    # Collect activations after input_layernorm
    with torch.no_grad():
        for act_id in range(len(activations)):
            act = block_fp.input_layernorm(activations[act_id].cuda())            
            activations[act_id] = act.cpu()
            
            act_q = block_q.input_layernorm(activations_q[act_id].cuda())            
            activations_q[act_id] = act_q.cpu()

    # Initialize q,k,v-projs
    H = prepare_hessian(activations)
    Hq = prepare_hessian_q(activations, activations_q)
    C = prepare_C(H, Hq)

    block_q_attn = block_q.self_attn
    block_fp_attn = block_fp.self_attn
    for layer_name in ["q_proj", "k_proj", "v_proj"]:
        layer_q = getattr(block_q_attn, layer_name)
        layer_fp = getattr(block_fp_attn, layer_name)
        configure_single_layer(layer_q, layer_fp, bits)
        if with_opt:
            optimize_quant_params(layer_q, layer_fp, bits, H, C)

    # Collect attention-out activations
    with torch.no_grad():
        for act_id in range(len(activations)):
            act = block_fp_attn.compute_attention(
                hidden_states=act.cuda(), 
                position_embeddings=position_embeddings,
                attention_mask=causal_mask
            )[0]
            activations[act_id] = act.cpu()
            
            act_q = block_q_attn.compute_attention(
                hidden_states=act_q.cuda(), 
                position_embeddings=position_embeddings,
                attention_mask=causal_mask
            )[0]
            activations_q[act_id] = act_q.cpu()

    # Initialize o_proj
    layer_q = block_q.self_attn.o_proj
    layer_fp = block_fp.self_attn.o_proj
    
    H = prepare_hessian(activations)
    Hq = prepare_hessian_q(activations, activations_q)
    C = prepare_C(H, Hq)

    configure_single_layer(layer_q, layer_fp, bits)
    if with_opt:
        optimize_quant_params(layer_q, layer_fp, bits, H, C)

    # Collect self_attn outs
    with torch.no_grad():
        for act_id in range(len(activations)):
            act = activations[act_id].cuda()
            res_act = residual_activations[act_id].cuda()
            activations[act_id] = (block_fp.self_attn.o_proj(act) + res_act).cpu()

            act_q = activations_q[act_id].cuda()
            res_act_q = residual_activations_q[act_id].cuda()
            activations_q[act_id] = (block_q.self_attn.o_proj(act_q) + res_act_q).cpu()


    ##### MLP #####

    # Copy activations for the residual stream
    residual_activations = [x.clone() for x in activations]
    residual_activations_q = [x.clone() for x in activations_q]

    # Collect activations after post_attention_layernorm
    with torch.no_grad():
        for act_id in range(len(activations)):
            act = block_fp.post_attention_layernorm(activations[act_id].cuda())
            activations[act_id] = act.cpu()

            act_q = block_q.post_attention_layernorm(activations_q[act_id].cuda())
            activations_q[act_id] = act_q.cpu()

    # Initialize gate_proj and up_proj
    H = prepare_hessian(activations)
    Hq = prepare_hessian_q(activations, activations_q)
    C = prepare_C(H, Hq)

    block_q_mlp = block_q.mlp
    block_fp_mlp = block_fp.mlp
    for layer_name in ["gate_proj", "up_proj"]:
        layer_q = getattr(block_q_mlp, layer_name)
        layer_fp = getattr(block_fp_mlp, layer_name)
        configure_single_layer(layer_q, layer_fp, bits)
        if with_opt:
            optimize_quant_params(layer_q, layer_fp, bits, H, C)

    # Collect internal mlp activations
    with torch.no_grad():
        for act_id in range(len(activations)):
            act = activations[act_id].cuda()
            act = block_fp_mlp.act_fn(block_fp.mlp.gate_proj(act)) * block_fp.mlp.up_proj(act)
            activations[act_id] = act.cpu()

            act_q = activations_q[act_id].cuda()
            act_q = block_q_mlp.act_fn(block_q.mlp.gate_proj(act_q)) * block_q.mlp.up_proj(act_q)
            activations_q[act_id] = act_q.cpu()

    # Initialize down_proj
    layer_q = block_q.mlp.down_proj
    layer_fp = block_fp.mlp.down_proj
    
    H = prepare_hessian(activations)
    Hq = prepare_hessian_q(activations, activations_q)
    C = prepare_C(H, Hq)

    configure_single_layer(layer_q, layer_fp, bits)
    if with_opt:
        optimize_quant_params(layer_q, layer_fp, bits, H, C)

    # Collect mlp outs
    with torch.no_grad():
        for act_id in range(len(activations)):
            act = activations[act_id].cuda()
            res_act = residual_activations[act_id].cuda()
            activations[act_id] = (block_fp.mlp.down_proj(act) + res_act).cpu()

            act_q = activations_q[act_id].cuda()
            res_act_q = residual_activations_q[act_id].cuda()
            activations_q[act_id] = (block_q.mlp.down_proj(act) + res_act_q).cpu()


def init_quant_model_hessian(model_q, model_fp, bits, dataloader):
    embed_tokens = model_fp.get_decoder().embed_tokens.cuda()
    embed_tokens_device = embed_tokens.weight.device

    _batch = next(iter(dataloader))
    _inputs_embeds = embed_tokens(_batch.to(embed_tokens_device))

    cache_position = torch.arange(_inputs_embeds.shape[1], device=_inputs_embeds.device)
    position_ids = cache_position.unsqueeze(0)
    causal_mask = create_causal_mask(
        config=model_fp.config,
        input_embeds=_inputs_embeds,
        attention_mask=None,
        cache_position=cache_position,
        past_key_values=None,
        position_ids=position_ids,
    )

    position_embeddings = model_fp.get_decoder().rotary_emb(_inputs_embeds, position_ids)

    # Prepare activations
    activations = []
    with torch.no_grad():
        for batch in dataloader:
            activations.append(embed_tokens(batch.to(embed_tokens_device)).cpu())
    activations_q = [a.clone() for a in activations]

    for decoder_layer_id in tqdm(range(len(model_q.get_decoder().layers))):
        # if decoder_layer_id > 2:
        #     break

        block_q = model_q.get_decoder().layers[decoder_layer_id].cuda()
        block_fp = model_fp.get_decoder().layers[decoder_layer_id].cuda()

        # init_quant_block_hessian(
        #     block_q,
        #     block_fp,
        #     bits,
        #     activations,
        #     causal_mask,
        #     position_embeddings,
        #     with_opt=True
        # )

        init_quant_block_hessian_2(
            block_q,
            block_fp,
            bits,
            activations,
            activations_q,
            causal_mask,
            position_embeddings,
            with_opt=True
        )

        block_q = block_q.cpu()
        block_fp = block_fp.cpu()


config = {
    "dataset_name" : "wiki",
    "split": "train[:10000]",
    # "dataset_name" : "slim_pajama",
    # "split": "train[:5000]",
    "seq_length": 4096,
    "n_seq" : 64, #128,
    "batch_size": 8,
    "random_seed": 'no_rand'
}
dataloader = qlib.QATDataset(
    config=config,
    tokenizer=tokenizer
).get_dataloader()
print(len(dataloader))
init_quant_model_hessian(qmodel, model, BITS, dataloader)

8


  0%|          | 0/32 [00:00<?, ?it/s]

220.8610076904297 -> 16.006084442138672
270.2167053222656 -> 17.31378936767578
158.06936645507812 -> 5.9089813232421875
0.3702833652496338 -> 0.03462900221347809
571.605224609375 -> 234.9871826171875
557.256103515625 -> 226.6083984375
5.793052673339844 -> 1.3856124877929688


  3%|▎         | 1/32 [00:52<27:15, 52.77s/it]

2806.46923828125 -> 742.6286010742188
2754.2802734375 -> 750.1689453125
506.09649658203125 -> 89.96592712402344
2.8629579544067383 -> 0.28932833671569824
1511.875 -> 805.513671875
1342.026611328125 -> 710.8385009765625
2970.34033203125 -> 8.761390686035156


...

29108.828125 -> 175.52877807617188
29412.76953125 -> 179.634033203125
26255.0859375 -> 156.2301025390625
609797.5 -> 1212.5782470703125
52327.2890625 -> 248.03414916992188
47561.96875 -> 223.94113159179688
4589.9091796875 -> 13.34028434753418


 94%|█████████▍| 30/32 [25:28<01:41, 50.88s/it]

32930.0859375 -> 167.1314697265625
32648.41015625 -> 167.80740356445312
29722.390625 -> 151.41702270507812
922815.5 -> 2012.63427734375
55767.578125 -> 283.00445556640625
50153.7734375 -> 244.27688598632812
579165056.0 -> -360064.375


 97%|█████████▋| 31/32 [26:23<00:51, 51.08s/it]


_LinAlgError: linalg.inv: The diagonal element 2534 is zero, the inversion could not be completed because the input matrix is singular.

In [ ]:
config = {
    "dataset_name" : "wiki",
    "split": "test",
    "seq_length": 2048,
    "batch_size": 1,
    "random_seed": 'no_rand'
}

dataloader = qlib.QATDataset(
    config=config,
    tokenizer=tokenizer
).get_dataloader()

qmodel = qmodel.cuda()
# qmodel = model.cuda()

with torch.no_grad():
    with torch.amp.autocast('cuda', dtype=torch.float16):
        res = qlib.evaluate(qmodel, dataloader, print_times=25)
        print(res)

  0%|          | 0/166 [00:00<?, ?it/s]W1228 18:25:51.958000 170414 site-packages/torch/_dynamo/convert_frame.py:1358] [0/8] torch._dynamo hit config.recompile_limit (8)
W1228 18:25:51.958000 170414 site-packages/torch/_dynamo/convert_frame.py:1358] [0/8]    function: 'reconstruct_weight' (/tmp/ipykernel_170414/2410632724.py:24)
W1228 18:25:51.958000 170414 site-packages/torch/_dynamo/convert_frame.py:1358] [0/8]    last reason: 0/7: tensor 'self._buffers['compressed_weight']' size mismatch at index 0. expected 11008, actual 4096
W1228 18:25:51.958000 170414 site-packages/torch/_dynamo/convert_frame.py:1358] [0/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W1228 18:25:51.958000 170414 site-packages/torch/_dynamo/convert_frame.py:1358] [0/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html
  4%|▍         | 7/166 [00:04<01:37,  1.62it/s]

112621.53783384823


  8%|▊         | 13/166 [00:07<01:32,  1.65it/s]

113769.37736770592


 11%|█▏        | 19/166 [00:10<01:28,  1.65it/s]

109991.26188648363


 15%|█▌        | 25/166 [00:13<01:25,  1.65it/s]

In [ ]:
### w3gs128

# base:            8.76
# H (wiki):        6.17 
# H (slim-pajama): 6.31 

In [ ]:
### w2gs64

# base:            9583
# H (wiki):        27.7
# H (slim-pajama): 36.6